In [ ]:
# Load necessary libraries
install.packages("factoextra")
library(readxl)
library(dplyr)
library(stringr)
library(lubridate)
library(tidyr)
library(factoextra)


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘rbibutils’, ‘Deriv’, ‘microbenchmark’, ‘Rdpack’, ‘doBy’, ‘SparseM’, ‘MatrixModels’, ‘minqa’, ‘nloptr’, ‘reformulas’, ‘RcppEigen’, ‘lazyeval’, ‘carData’, ‘Formula’, ‘pbkrtest’, ‘quantreg’, ‘lme4’, ‘crosstalk’, ‘estimability’, ‘mvtnorm’, ‘numDeriv’, ‘corrplot’, ‘viridis’, ‘car’, ‘DT’, ‘ellipse’, ‘emmeans’, ‘flashClust’, ‘leaps’, ‘multcompView’, ‘scatterplot3d’, ‘ggsci’, ‘cowplot’, ‘ggsignif’, ‘gridExtra’, ‘polynom’, ‘rstatix’, ‘plyr’, ‘abind’, ‘dendextend’, ‘FactoMineR’, ‘ggpubr’, ‘reshape2’, ‘ggrepel’



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union


Loading required package: ggplot2

Welcome! Wan

In [ ]:
install.packages("openxlsx")  # Only once
library(openxlsx)

In [ ]:
df <- read_excel("/content/Monthly Booking Details.xlsx")

# Replace "-" with "Not specified"
df <- df %>% mutate(across(everything(), ~replace(., . == "-", "Not specified")))

# Convert dates
df <- df %>%
  mutate(
    `Check-in` = dmy(`Check-in`),
    `Check out` = dmy(`Check out`)
  )

# Add 'Month' column
df <- df %>% mutate(Month = format(`Check-in`, "%Y-%m"))

# Clean 'Total price' and ensure numeric values
df <- df %>%
  mutate(
    `Total price` = as.numeric(str_replace_all(as.character(`Total price`), "[^0-9.]", "")),
    `Nights Count` = as.numeric(`Nights Count`)
  )

# Trim white space in names
df <- df %>% mutate(`Booking Name` = str_trim(`Booking Name`))

# STEP 2: Currency Conversion --------------------------------------

# Example conversion table (adjust as needed)
conversion_rates <- data.frame(
  Currency = c("E£", "€", "ر.س"),
  Rate_to_EGP = c(1, 53, 16)
)

# Join conversion rates
df <- df %>%
  left_join(conversion_rates, by = "Currency")

# Create new column for converted total price
df <- df %>%
  mutate(`Total price EGP` = `Total price` * Rate_to_EGP)

# STEP 3: Aggregate Per Customer -----------------------------------

customer_df <- df %>%
  group_by(`Booking Name`) %>%
  summarise(
    TotalSpend = sum(`Total price EGP`, na.rm = TRUE),
    AvgNights = mean(`Nights Count`, na.rm = TRUE),
    BookingCount = n()
  ) %>%
  ungroup() %>%
  replace_na(list(TotalSpend = 0, AvgNights = 0, BookingCount = 0))

# STEP 4: K-Means Clustering ---------------------------------------

# Scale features
scaled_features <- scale(customer_df[, c("TotalSpend", "AvgNights", "BookingCount")])

# Elbow plot (optional)
# fviz_nbclust(scaled_features, kmeans, method = "wss")

# Run K-means with K = 3
set.seed(123)
kmeans_result <- kmeans(scaled_features, centers = 3, nstart = 25)
customer_df$Cluster <- as.factor(kmeans_result$cluster)

# STEP 5: Add Descriptive Segment Labels ---------------------------

customer_df <- customer_df %>%
  mutate(Segment = case_when(
    Cluster == 1 ~ "High Value Loyalists",
    Cluster == 2 ~ "Budget Frequent",
    Cluster == 3 ~ "Low Engagement"
  ))

# Optional: Add Segment Description for tooltips
customer_df <- customer_df %>%
  mutate(Segment_Description = case_when(
    Segment == "High Value Loyalists" ~ "Frequent, long-stay, high-spending customers.",
    Segment == "Budget Frequent" ~ "Loyal but price-sensitive customers who book often.",
    Segment == "Low Engagement" ~ "Low-activity or one-time customers."
  ))

# STEP 6: Merge Back to Booking-Level Data --------------------------

df_final <- df %>%
  left_join(customer_df, by = "Booking Name")

# STEP 7: Export Final Data ----------------------------------------

write.xlsx(df_final, file = "Monthly_Booking_with_Segments_CurrencyHandled.xlsx", overwrite = TRUE)

# Optional: Print cluster summary
cluster_summary <- customer_df %>%
  group_by(Segment) %>%
  summarise(
    Avg_TotalSpend = round(mean(TotalSpend), 2),
    Avg_AvgNights = round(mean(AvgNights), 2),
    Avg_BookingCount = round(mean(BookingCount), 2),
    Count = n()
  )

print(cluster_summary)

# ---------------------------------------------------------
# 💡 This version preserves original 'Total price' and 'Currency',
#     adds 'Total price EGP' for fair financial comparison,
#     and keeps the dashboard Tableau-ready.
# ---------------------------------------------------------


ERROR: Error in read_excel("/content/Monthly Booking Details.xlsx"): could not find function "read_excel"
